In [2]:
import tensorflow as tf

class GP038:
    def __init__(self):
        try:
            self.__model = tf.keras.models.load_model('GP038.keras')
            print("Model loaded")
            self.__model.summary()
        except:
            print("Load error")
        
    def __padDat(self, data):
        if len(data[0]) <= 1294:
            ...
    
    def predCheat(self, data):
        data = self.__padDat(data)
        prediction = self.__model.predict(data)

GP038 = GP038()

Model loaded
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_30 (Conv1D)          (None, 1294, 12)          444       
                                                                 
 average_pooling1d_30 (Avera  (None, 647, 12)          0         
 gePooling1D)                                                    
                                                                 
 conv1d_31 (Conv1D)          (None, 647, 12)           444       
                                                                 
 average_pooling1d_31 (Avera  (None, 323, 12)          0         
 gePooling1D)                                                    
                                                                 
 lstm_15 (LSTM)              (None, 12)                1200      
                                                                 
 flatten_15 (Flatten)        (None, 12) 